# Лабораторная работа по Теории кодирования
## Лабораторная работа №1
Выполнили студенты группы 6401-010302D

Баловнева Юлия

Богатырев Дмитрий


In [2]:
import numpy as np
import itertools

def mult(m1, m2):
    A = np.array(m1, dtype=int)
    B = np.array(m2, dtype=int)
    C = (A@B)%2
    return np.array(C, dtype=bool)

def code_length(matr):
    d = len(matr[0])
    for i in range(len(matr)-1):
        for j in range(i+1,len(matr)):
            d = min(d,sum(np.bitwise_xor(matr[i],matr[j])))
    return d

In [3]:
class LinearCode:
    ''' Класс линейных кодов.

    Атрибуты:\n
    matr: копия исходной матрицы\n
    k: количество строк матрицы\n
    n: количество столбцов матрицы
    
    '''
    # Конструктор
    def __init__(self, n_array):
        self.matr = np.array(n_array, dtype=bool)
        self.n = n_array.shape[1]
        self.k = n_array.shape[0]
    
    def find_first(self, line):
        '''Поиск первого ненулевого элемента в строке.
        
        :param:
        line: строка матрицы.
        
        :return: h: возвращаем позицию первого ненулевого элемента в строке. 
        '''
        # Задаем h = количеству элементов в сторке
        h = self.n
        # В строке line перебираем элементы пока не натолкнемся на 1
        for i in range(self.n):
            if self.matr[line,i]:
                h = i
                break
        # Возвращаем позицию найденной единицы в строке
        return h
    
    def xor_swap(self, line1, line2):
        '''Меняет две строки местами с помощью поэлементного XOR.
        
        line1: первая строка.\n
        line2: вторая строка.

        '''
        # Поэлементный XOR 
        for w in range(self.n):
            self.matr[line1,w] ^= self.matr[line2,w]
            self.matr[line2,w] ^= self.matr[line1,w]
            self.matr[line1,w] ^= self.matr[line2,w]


    def sum(self, line1, line2):
        '''Складывает первую переданную строку со второй.
        
        line1: первая строка.\n
        line2: вторая строка.
        '''
        # Поэлементный XOR 
        for w in range(self.n):
            self.matr[line2,w] ^= self.matr[line1,w]

    def __str__(self):
        '''Собирает строку (смена типа от bool к string).

        :return: s: строка с матрицей.            
        '''
        s = ''
        for i in range(self.k):
            s += '['
            for j in range(self.n):
                if self.matr[i,j]: s += "1 "
                else: s += "0 "
            s += "\b]\n"
        return s
    
    def fillrand(self):
        '''Заполняет матрицу случайным образом нулями и единицами.
        
        '''
        rand = np.random
        for i in range(self.k):
            for j in range(self.n):
                self.matr[i,j] = np.round(rand.random())

    def sort(self):
        '''Сортировка строк матрицы.
        '''
        # sort
        for i in range(self.k):
            for j in range(self.k-i-1):
                if self.find_first(j) > self.find_first(j+1):
                    self.xor_swap(j, j+1)

    def ref(self):
        '''Приводит матрицу к ступенчатому виду.
        
            ---
            Ступенчатой матрица - такая матрица, что
            * все ненулевые строки располагаются над всеми чисто нулевыми строками
            * ведущий элемент (первый, считая слева направо, ненулевой элемент строки)
            каждой ненулевой строки располагается строго правее ведущего элемента в
            строке, расположенной выше данной.
            ---
        
        
        '''
        # Проходит по всем строкм матрицы
        for i in range(self.k-1):
            # Сравнивает рабочую строку с текущей
            for j in range(i+1, self.k):
                # Запоминаем первые элементы
                first_i = self.find_first(i)
                first_j = self.find_first(j)
                # Если они равны, то делаем xor со строкой
                # Поскольку рабочая строка всегда ниже текущей, то при xor первый элемент всех нижних строк зануляется
                if first_i == first_j:
                    self.sum(i,j)
                # Если встретили строку, у которой первый элемент раньше чем у текущей, то меняем строки местами
                elif first_i > first_j:
                    self.xor_swap(i,j)
                # Иначе все хорошо, идем дальше
        # Удаляем занулившиеся строки, которые могли бы получится
        self.delete_redundant_lines()
        

    def rref(self):
        '''Приводит матрицу к приведенному ступенчатому виду. 
        
            ---
            Ступенчатая матрица называется приведенной, если матрица,  не имеет нулевых строк, 
            и все ведущие элементы ее строк равны единице. При этом все элементы основных столбцов, 
            помимо ведущих элементов, являются нулями.
            
        '''
        # Сначала приводим матрицу к ступенчатому виду
        self.ref()
        # Дальше для всех строк...
        for i in range(1,self.k):
            # ...мы ищем номер ведущего элемента строки...
            ist = self.find_first(i)
            # ...и делаем xor для всех строк выше текущей
            for j in range(i-1,-1,-1):
                if(self.matr[j,ist]):
                    self.sum(i,j)
            # Таким образом зануляются все элементы над ведущим
        # Удаляем занулившиеся строки, которые могли бы получится
        self.delete_redundant_lines()
    

    def delete_redundant_lines(self):
        '''Удаляет нулевые строки из матрицы.
        '''
        # Копия матрицы, с которой ведется работа
        temp = self.matr.copy()
        offset = 0 # сдвиг
        for i in range(self.k):
            # Если номер первой единицы в строке == кол-ву элементов самой строки (то есть единиц там нет)
            # Удаляем эту строку и сдвигаем указатель на 1
            if self.find_first(i) == self.n:
                temp = np.delete(temp, i - offset, 0)
                offset += 1
        # Перезаписываем матрицу на новую и очищенную
        self.matr = temp
        # Обновляем переменные-размеры матрицы
        self.n = temp.shape[1]
        self.k = temp.shape[0]

                
    def create_verif_matr(self):
        '''Создает проверочную матрицу на основе порождающей.
        
        :return: LinearCode(H): проверочная матрица. 
        '''
        # Создаем новую матрицу в приведенном ступенчатом виде на основе текущей
        rrefed_matr = LinearCode(self.matr.copy())
        rrefed_matr.rref()

        # Определяем индексы ведущих элементов
        leads = np.array([rrefed_matr.find_first(i) for i in range(rrefed_matr.k)])

        # Создаем сокращенную матрицу 
        reduced_matr = np.zeros(shape=(rrefed_matr.k,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        
        # Заполняем ее значениями из ступенчатой
        for i in range(rrefed_matr.k):
            offset = 0 # сдвиг
            for j in range(rrefed_matr.n):
                # Если в столбце содержится ведущий элемент, то его пропускаем (и увеличиваем сдвиг на 1)
                # Иначе добавляем этот элемент в сокращенную матрицу
                if j in leads: 
                    offset+=1
                else:
                    reduced_matr[i,j-offset] = rrefed_matr.matr[i,j]

        # Единичная матрица
        Ind = np.identity(n = rrefed_matr.n-rrefed_matr.k, dtype = bool)

        # Создаем проверочную матрицу
        H = np.zeros(shape=(rrefed_matr.n,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        # сдвиги
        offset_i = 0
        offset_matr = 0
        # Заполняем ее значениями
        for i in range(H.shape[0]):
            # Если номер строки содержится в массиве с ведущими элементами, то заполняем строку из сокращенной матрицы
            # И прибавляем 1 к сдвигу для элементов единичной матрицы
            if i in leads:
                for j in range(H.shape[1]):
                    H[i,j] = reduced_matr[i-offset_matr,j]
                offset_i += 1
            # Иначе заполняем строку из единичной матрицы
            # И прибавляем 1 к сдвику для матрицы сокращенной
            else:
                for j in range(H.shape[1]):
                    H[i,j] = Ind[i-offset_i,j]
                offset_matr += 1
        
        return LinearCode(H)
    
    def gen_codewords_summing(self):
        wordset = set()
        wordset.add(tuple((np.zeros(self.n, dtype=bool))))
        for i in range(1, self.k+1):
            combinations = np.array(list(itertools.combinations(range(self.k), i)))
            for comb in combinations:
                word = np.zeros(self.n, dtype=bool)
                for j in comb:
                    word ^= self.matr[j]
                wordset.add(tuple(word.tolist()))
        return wordset
    
    def gen_codewords_bin(self):
        wordset = set()
        binset = set(tuple(itertools.product((True,False),repeat=self.k)))
        for bin in binset:
            word = tuple((mult(bin,self.matr)).tolist())
            wordset.add(word)
        return wordset



In [4]:
s = np.array([[1,0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
[0, 0, 0, 1 ,1 ,1 ,0, 1, 0, 1, 0],
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
[1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
[0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]])
arr2 = LinearCode(s)

# arr2 = LinearCode(np.zeros(shape=[4,6],dtype=bool))
# arr2.fillrand()
print("S = ")
print(arr2)
H = arr2.create_verif_matr()

print("G = ")
arr2.ref()
print(arr2)

print("G* = ")
arr2.rref()
print(arr2)

print("H = ")
print(H)

S = 
[1 0 1 1 0 0 0 1 0 0 1]
[0 0 0 1 1 1 0 1 0 1 0]
[0 0 0 0 1 0 0 1 0 0 1]
[1 0 1 0 1 1 1 0 0 0 1]
[0 0 0 0 1 0 0 1 1 1 0]
[1 0 1 1 1 0 0 0 0 0 0]

G = 
[1 0 1 1 0 0 0 1 0 0 1]
[0 0 0 1 1 1 0 1 0 1 0]
[0 0 0 0 1 0 0 1 0 0 1]
[0 0 0 0 0 0 1 0 0 1 0]
[0 0 0 0 0 0 0 0 1 1 1]

G* = 
[1 0 1 0 0 1 0 1 0 1 0]
[0 0 0 1 0 1 0 0 0 1 1]
[0 0 0 0 1 0 0 1 0 0 1]
[0 0 0 0 0 0 1 0 0 1 0]
[0 0 0 0 0 0 0 0 1 1 1]

H = 
[0 1 1 1 1 0]
[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 1 1]
[0 0 0 1 0 1]
[0 0 1 0 0 0]
[0 0 0 0 1 0]
[0 0 0 1 0 0]
[0 0 0 0 1 1]
[0 0 0 0 1 0]
[0 0 0 0 0 1]



In [5]:
print("Кодовые слова, сгенерированные путем сложения слов порождающего множества:")
code_sum = arr2.gen_codewords_summing()
print(code_sum)
print("Кодовые слова, сгенерированные путем умножения всех двоичных слов на G:")
code_bin = arr2.gen_codewords_bin()
print(code_bin)

print("Равны ли эти множества слов: ")
print(code_sum == code_bin)

print("Умножение всех слов на H дает нулевые вектора: ")
for i in code_bin:
    check = mult(i,H.matr)
    print(check)

print("Кодовое расстояние: ")
print("d =", code_length(list(code_sum)))
print("Кратность обнаруживаемой ошибки: ")
print("t =", code_length(list(code_sum))-1)

Кодовые слова, сгенерированные путем сложения слов порождающего множества:
{(False, False, False, False, False, False, True, False, False, True, False), (False, False, False, True, True, True, True, True, True, True, True), (True, False, True, False, True, True, False, False, True, False, False), (False, False, False, False, False, False, True, False, True, False, True), (True, False, True, True, True, False, True, False, False, True, False), (True, False, True, True, False, False, True, True, True, False, False), (False, False, False, False, True, False, False, True, True, True, False), (False, False, False, True, True, True, False, True, False, True, False), (False, False, False, True, True, True, True, True, False, False, False), (False, False, False, False, True, False, True, True, True, False, False), (True, False, True, True, False, False, True, True, False, True, True), (False, False, False, True, True, True, False, True, True, False, True), (True, False, True, False, False, Tru

In [6]:
print("Исходное кодовое слово: ")
v = np.array([1,0,1,1,1,0,1,0,0,1,0], dtype = int)
print(v)

print("Ошибка кратности 1:")
e1 = np.array([0,0,1,0,0,0,0,0,0,0,0], dtype = int)
print(e1)
print("Ошибка кратности 2:")
e2 = np.array([0,0,0,0,0,0,1,0,0,1,0], dtype = int)
print(e2)

print("Умножаем на ошибку e1, она обнаруживается:")
v1 = v^e1
print(mult(v1,H.matr))
print("Умножаем на ошибку e2, она не обнаруживается:")
v2 = v^e2
print(mult(v2,H.matr))


Исходное кодовое слово: 
[1 0 1 1 1 0 1 0 0 1 0]
Ошибка кратности 1:
[0 0 1 0 0 0 0 0 0 0 0]
Ошибка кратности 2:
[0 0 0 0 0 0 1 0 0 1 0]
Умножаем на ошибку e1, она обнаруживается:
[False  True False False False False]
Умножаем на ошибку e2, она не обнаруживается:
[False False False False False False]
